<a href="https://colab.research.google.com/github/yuyuponponzu/signate_assesment/blob/master/assesment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **セットアップ**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
%cd /content/drive/'My Drive'/SIGNATE/
!ls

/content/drive/My Drive/SIGNATE
assesment.ipynb  sample_submit.csv  test.csv   train.pkl
catboost_info	 target.pkl	    train.csv  zipcode_onehot.pkl


In [2]:
!pip install japanize-matplotlib
#!pip install pymc3==3.8
!pip install pandas-profiling==2.7.1

# **データ探索**

In [3]:
import pandas as pd
import warnings
import japanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from IPython.core.display import display, HTML 
warnings.filterwarnings('ignore')
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

import pandas_profiling as pdp
df=pd.concat([train,test],sort=False)

/usr/local/lib/python3.6/dist-packages/japanize_matplotlib/japanize_matplotlib.py:15: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.
  font_list = font_manager.createFontList(font_files)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
train.head()

,id,accommodates,amenities,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,thumbnail_url,zipcode,y
0,0,6,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",2.0,Real Bed,1.0,4.0,flexible,LA,t,My place is meant for family and a few friends...,2016-07-27,t,f,NaN,2016-07-13,f,2016-07-27,33.788931,-118.154761,The Penthouse,NaN,1,Apartment,60.0,Private room,NaN,90804,138.0
1,1,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,DC,t,This is a new listing for a lovely guest bedro...,2016-09-12,t,t,100%,2015-12-30,f,2017-03-31,38.934810,-76.978190,Guest Bedroom in Brookland,Brookland,9,House,100.0,Private room,https://a0.muscache.com/im/pictures/e4d8b51f-6...,20018,42.0
2,2,2,"{TV,Internet,""Wireless Internet"",Kitchen,""Indo...",2.0,Real Bed,1.0,1.0,strict,NYC,t,We're looking forward to your stay at our apt....,2016-06-15,t,f,100%,2016-05-21,t,2017-08-13,40.695118,-73.926240,Clean Modern Room in Lux Apt 1 Block From J Train,Bushwick,27,Apartment,83.0,Private room,https://a0.muscache.com/im/pictures/5ffecc9b-d...,NaN,65.0
3,3,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,SF,t,BEST CITY VIEWS - - ROOF DECK W/ BBQ & WiFi - ...,2014-03-15,t,t,100%,2012-06-19,t,2017-09-03,37.796728,-122.411906,BEST views + reviews! 5/5 stars*****,Nob Hill,38,Apartment,95.0,Private room,NaN,94133,166.0
4,4,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,Real Bed,1.0,1.0,strict,NYC,t,Charming Apartment on the upper west side of M...,2015-08-05,t,t,100%,2015-03-25,f,2017-09-10,40.785050,-73.974691,Charming 1-bedroom - UWS Manhattan,Upper West Side,5,Apartment,100.0,Entire home/apt,https://a0.muscache.com/im/pictures/92879730/5...,10024,165.0


In [6]:
test.head()

,id,accommodates,amenities,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,thumbnail_url,zipcode
0,0,6,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",2.0,Real Bed,2.0,2.0,strict,Boston,t,Feel free to book INSTANTLY. You can check-in ...,2017-01-09,t,f,100%,2016-08-23,t,2017-09-25,42.359278,-71.069962,Gorgeous 2BR/2BA Duplex in Beacon Hill,Beacon Hill,58,House,90.0,Entire home/apt,https://a0.muscache.com/im/pictures/7e4808b4-5...,02114
1,1,3,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,moderate,LA,t,The guest house is close to: Equinox West Holl...,2016-08-17,t,t,100%,2014-09-03,f,2017-05-02,34.084747,-118.367355,Luxury 1 Bedroom West Hollywood City Center,West Hollywood,4,Guesthouse,100.0,Entire home/apt,https://a0.muscache.com/im/pictures/5392fbd6-6...,90046
2,2,2,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",1.0,Real Bed,0.0,1.0,flexible,NYC,f,Private room in a three bedroom apartment in N...,NaN,t,t,100%,2012-10-17,f,NaN,40.720541,-73.959192,Bedroom with Patio in Prime Williamsburg Locat...,Williamsburg,0,Apartment,NaN,Private room,https://a0.muscache.com/im/pictures/544d3b89-d...,11249.0
3,3,4,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,2.0,strict,NYC,f,The apartment is located in historic Bed Stuy ...,NaN,t,t,NaN,2013-01-23,f,NaN,40.681117,-73.944091,Cozy apartment in Brooklyn,Bedford-Stuyvesant,0,Apartment,NaN,Entire home/apt,https://a0.muscache.com/im/pictures/26baf7ba-0...,11216
4,4,3,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.5,Real Bed,1.0,2.0,strict,LA,t,"Our cozy, pet friendly one bedroom apartment/l...",2015-08-01,t,t,100%,2014-12-28,f,2016-09-11,34.150995,-118.409359,"Cozy, sunny, pet friendly loft/apt",NaN,6,Loft,92.0,Entire home/apt,https://a0.muscache.com/im/pictures/86107545/9...,91604


In [7]:
print("!!!!!!!!!!!!!!!!!!!!!!!!!!TRAIN")
print("COLUMNS")
display(train.columns)
print("LENGTH")
display(train.shape)

print("")
print("!!!!!!!!!!!!!!!!!!!!!!!!!!TEST")
print("COLUMNS")
display(test.columns)
print("LENGTH")
display(test.shape)

print("")
print("!!!!!!!!!!!!!!!!!!!!!!!!!!NULL CHECK")
display(df.isnull().sum())

print("")
print("!!!!!!!!!!!!!!!!!!!!!!!!!!UNIQUE CHECK")
for i in df.columns:
    # if (i in ["id", "host_since", "last_review", "latitude", "longitude", "name", "neighbourhood", "thumbnail_url", "zipcode", "y", "description" ,"first_review"]):
    #   continue;
    if i == "amenities":
      print("--------------------"+ i + "--------------------")
      func = lambda x: x.replace("\'","").replace("{","").replace("}","").replace('\"',"").split(",")
      s = set()
      for i in df["amenities"]:
        s = s | set(func(i))  
      print(s)
    else :
      print("--------------------"+ i + "--------------------")
      print(df[i].unique())

SyntaxError: ignored

In [ ]:
# df[df["y"] > 750]
#2%弱が外れ値？

#yの分布を見てみる
df["y"].hist()
df[df["y"] > 750]["y"].mean()

In [ ]:
# レビュースコアとyの値の関係
# ついでにyの外れ値の具合を見る
# これを見る感じ、ガッツリ気色悪い外れ値はなさそう
temp = df.copy(deep=True)
temp["review_scores_rating"] = temp["review_scores_rating"].map(lambda x: 0 if np.isnan(x) else round(x))
sns.stripplot(x="review_scores_rating",y="y",data=temp)#正の相関

In [ ]:
temp["review_scores_rating"].unique()

In [ ]:
pdp.ProfileReport(df)

# **とりあえずベースライン**

In [4]:
!pip install catboost scikit-learn

In [5]:
from sklearn.model_selection import KFold
# from catboost import CatBoost
from catboost import CatBoostRegressor, FeaturesData, Pool
from sklearn.metrics import mean_squared_error

## **na全て削除、共通化できそうにない特徴量も全て削除**

In [ ]:
# results
# [85.9447521058982, 78.32003890836233, 82.52707618223008, 86.6723659654161, 84.45736221190403]
def cross_validate(split_size=5):
  results = []
  kf = KFold(n_splits=split_size)
  for train_idx, val_idx in kf.split(X_train, y_train):
    train_x = X_train[train_idx]
    train_y = y_train[train_idx]
    val_x = X_train[val_idx]
    val_y = y_train[val_idx]
    rmse = run_catboost(train_x, train_y, val_x, val_y)
    results.append(rmse)
  return results

def run_catboost(x,y,val_x,val_y):
  # params = {
  #       # タスク設定と損失関数
  #       'loss_function': 'Logloss',
  #       # 学習ラウンド数
  #       'num_boost_round': 100,
  #   }
  # モデルを学習する
  categorical_features_indices = []
  for index, j in enumerate(X_train[0]):
    if (type(j) == str):
      categorical_features_indices.append(index)
  train_pool = Pool(x, label=y, cat_features=categorical_features_indices)
  test_pool = Pool(val_x, label=val_y, cat_features=categorical_features_indices)
  model = CatBoostRegressor(iterations=3000, learning_rate=0.05, depth=5)
  model.fit(train_pool)
  # 検証用データを分類する
  # NOTE: 確率がほしいときは prediction_type='Probability' を使う
  pred_y = model.predict(test_pool)
  # 精度 (Accuracy) を検証する
  rmse = np.sqrt(mean_squared_error(val_y, pred_y))
  print('RMSE:', rmse)
  return rmse

X_train = df.dropna().drop(["amenities", "y","id","description", "host_since", "last_review", "latitude", "longitude", "name", "neighbourhood", "thumbnail_url", "zipcode","first_review"],axis=1).values
y_train = df.dropna()["y"].values
base_results = cross_validate(split_size=5)
print("results")
print(base_results)

In [ ]:
for i in X_train[0]:
  print(i)

In [ ]:
from sklearn.metrics import mean_squared_error

# **特徴量作成**

## **特徴量整形**

In [ ]:
#方向性
"""
まだ：
description 
　bertかなんかで特徴量算出、
　pcaとかで8割限度に圧縮
name
　一旦中身見てみて、意味有りげならbert
neighborhood
　中身見てみて、意味有りげならカテゴリ化
　もしくはbert

やった：
ammenities 
　ワンホット
bed_type 
　ワンホット
cancellation_policy 
　意味ある数を付与
city 
　ワンホット
first_reviewとlast_review 
　ある程度の期間(半年とか？)で区切ってカ
　テゴリ化してワンホットに(カテゴリのま
　まだと意味が取れない気がする)
host_since
　ワンホット
latitudeとlongitude
　それぞれ5区切りとか適切な範囲でカテゴ
　リ化、ワンホット化？zipcodeと相関高そ
　う
zipcode
　上3桁とかで括ってみるとか？これあれば
　上要らない気がする
review_scores_rating
　0.5区切りとかで集約
room_type
　カテゴリ化、順序付けできそう

property_type
　中身見てみて、意味有りげならカテゴリ化
　もしくはbert
thumbnail_url
　バイナリとか？有る無しだけで良い気がす
　る、もしくは何個有るかの数
"""

In [6]:
df_dummy = df.copy(deep=True)

In [ ]:
#amenities ワンホット
func = lambda x: x.replace("\'","").replace("{","").replace("}","").replace('\"',"").strip().split(",")
s = set()
for i in df["amenities"]:
  s = s | set(func(i))  
s.remove("")
s.add("id")
# print(s)

# from sklearn.preprocessing import LabelEncoder
# #LabelEncoderのインスタンスを生成
# le = LabelEncoder()
# #ラベルを覚えさせる
# le = le.fit(list(s))
# #ラベルを整数に変換
# le.transform(df['amenities'])

ameni_df = pd.DataFrame(columns=list(s))
for id,i in zip(df["id"],df["amenities"]):
  print(id)
  ret = i.replace("\'","").replace("{","").replace("}","").replace('\"',"").strip().split(",")
  newline = pd.DataFrame(np.zeros(len(list(s))).reshape(1,131), columns=list(s))
  newline["id"] = id
  ameni_df = ameni_df.append(newline, ignore_index=True)
  for _ in ret:
    ameni_df[_] = 1
ameni_df.head()

In [ ]:
"""
#bed_type 　ワンホット
df_dummy = df.copy(deep=True)
pd.get_dummies(df_dummy, columns=['bed_type'])
df_dummy.head()
"""

In [7]:
#cancellation_policy 　意味ある数を付与
cancellation_policy_dict = {'flexible': 5, 'strict':3, 'moderate':4, 'super_strict_30':2, 'super_strict_60':1}
df_dummy = df_dummy.replace(cancellation_policy_dict)
df_dummy.head()

,id,accommodates,amenities,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,thumbnail_url,zipcode,y
0,0,6,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",2.0,Real Bed,1.0,4.0,5,LA,t,My place is meant for family and a few friends...,2016-07-27,t,f,NaN,2016-07-13,f,2016-07-27,33.788931,-118.154761,The Penthouse,NaN,1,Apartment,60.0,Private room,NaN,90804,138.0
1,1,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,3,DC,t,This is a new listing for a lovely guest bedro...,2016-09-12,t,t,100%,2015-12-30,f,2017-03-31,38.934810,-76.978190,Guest Bedroom in Brookland,Brookland,9,House,100.0,Private room,https://a0.muscache.com/im/pictures/e4d8b51f-6...,20018,42.0
2,2,2,"{TV,Internet,""Wireless Internet"",Kitchen,""Indo...",2.0,Real Bed,1.0,1.0,3,NYC,t,We're looking forward to your stay at our apt....,2016-06-15,t,f,100%,2016-05-21,t,2017-08-13,40.695118,-73.926240,Clean Modern Room in Lux Apt 1 Block From J Train,Bushwick,27,Apartment,83.0,Private room,https://a0.muscache.com/im/pictures/5ffecc9b-d...,NaN,65.0
3,3,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,3,SF,t,BEST CITY VIEWS - - ROOF DECK W/ BBQ & WiFi - ...,2014-03-15,t,t,100%,2012-06-19,t,2017-09-03,37.796728,-122.411906,BEST views + reviews! 5/5 stars*****,Nob Hill,38,Apartment,95.0,Private room,NaN,94133,166.0
4,4,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,Real Bed,1.0,1.0,3,NYC,t,Charming Apartment on the upper west side of M...,2015-08-05,t,t,100%,2015-03-25,f,2017-09-10,40.785050,-73.974691,Charming 1-bedroom - UWS Manhattan,Upper West Side,5,Apartment,100.0,Entire home/apt,https://a0.muscache.com/im/pictures/92879730/5...,10024,165.0


In [ ]:
"""
#city 　ワンホット
pd.get_dummies(df_dummy, columns=['city'])
df_dummy.head()
"""

In [8]:
#first_review 日付に区切ってカテゴリ化
df_dummy['first_review'] = pd.to_datetime(df_dummy['first_review'])
# df_dummy['first_review'] = df_dummy['first_review'].dt.round("Y")
df_dummy['first_review'] = df_dummy['first_review'].dt.to_period('6M').dt.to_timestamp()

In [9]:
#last_review 日付に区切ってカテゴリ化
df_dummy['last_review'] = pd.to_datetime(df_dummy['last_review'])
df_dummy['last_review'] = df_dummy['last_review'].dt.to_period('6M').dt.to_timestamp()

In [10]:
#host_since 日付に区切ってカテゴリ化
df_dummy['host_since'] = pd.to_datetime(df_dummy['host_since'])
df_dummy['host_since'] = df_dummy['host_since'].dt.to_period('6M').dt.to_timestamp()

In [11]:
df_dummy = df_dummy.astype({'first_review': str, 'last_review': str, 'host_since': str})

In [12]:
#latitudeとlongitude　それぞれ5区切りとか適切な範囲でカテゴリ化、ワンホット化？zipcodeと相関高そう
print(df_dummy["latitude"].max())
print(df_dummy["latitude"].min())
df_dummy["latitude"] = df_dummy["latitude"].round()
df_dummy = df_dummy.astype({'latitude': str})

42.39043717872241
33.33890467150096


In [13]:
#latitudeとlongitude　それぞれ5区切りとか適切な範囲でカテゴリ化、ワンホット化？zipcodeと相関高そう
print(df_dummy["longitude"].max())
print(df_dummy["longitude"].min())
df_dummy["longitude"] = df_dummy["longitude"].round()
df_dummy["longitude"] = pd.cut(df_dummy["longitude"], 13, labels=[i for i in range(int(abs(df_dummy["longitude"].max())),
                  int(abs(df_dummy["longitude"].min())),4)])
df_dummy = df_dummy.astype({'longitude': str})

-70.98504659974512
-122.51149998987214


In [188]:
import math
#zipcode 頭3桁でワンホット
__1 = ["zip1_" + str(i) for i in range(10)]
__1.append("id")
zip1 = pd.DataFrame(columns=__1)
__2 = ["zip2_" + str(i) for i in range(10)]
__2.append("id")
zip2 = pd.DataFrame(columns=__2)
__3 = ["zip3_" + str(i) for i in range(10)]
__3.append("id")
zip3 = pd.DataFrame(columns=__3)

for id, _ in zip(df_dummy["id"],df_dummy["zipcode"]):
  if type(_) == str:
    _ = _[0:5]
  newline1 = pd.DataFrame(np.zeros(11).reshape(1,11), columns=__1)
  newline2 = pd.DataFrame(np.zeros(11).reshape(1,11), columns=__2)
  newline3 = pd.DataFrame(np.zeros(11).reshape(1,11), columns=__3)
  newline1["id"] = id
  newline2["id"] = id
  newline3["id"] = id
  try:
    if math.isnan(float(_)) :
      zip1 = zip1.append(newline1, ignore_index=True)
      zip2 = zip2.append(newline2, ignore_index=True)
      zip3 = zip3.append(newline3, ignore_index=True)
    else :  
      newline1[_[0]] = 1
      newline2[_[1]] = 1
      newline3[_[2]] = 1
      zip1 = zip1.append(newline1, ignore_index=True)
      zip2 = zip2.append(newline2, ignore_index=True)
      zip3 = zip3.append(newline3, ignore_index=True)
  except:
    zip1 = zip1.append(newline1, ignore_index=True)
    zip2 = zip2.append(newline2, ignore_index=True)
    zip3 = zip3.append(newline3, ignore_index=True)

zip1 = zip1.astype('str')
zip2 = zip2.astype('str')
zip3 = zip3.astype('str')

In [208]:
zip1 = zip1.drop(["zip1_" + str(i) for i in range(10)], axis=1).replace({"nan":0, "1.0":"1"})
zip2 = zip2.drop(["zip2_" + str(i) for i in range(10)], axis=1).replace({"nan":0, "1.0":"1"})
zip3 = zip3.drop(["zip3_" + str(i) for i in range(10)], axis=1).replace({"nan":0, "1.0":"1"})

In [247]:
# zip_ = pd.concat([zip1,zip2],axis=1)
# zip = pd.concat([zip_,zip3],axis=1)
# suffixes=('_zip1', '_zip2'),
# zip1=zip1.rename(columns={str(i):"zip1_"+str(i) for i in range(10)})
# zip2=zip2.rename(columns={str(i):"zip2_"+str(i) for i in range(10)})
# zip3=zip3.rename(columns={str(i):"zip3_"+str(i) for i in range(10)})
__ = pd.concat([zip1,zip2.drop("id",axis=1)], axis=1)
zip = pd.concat([__,zip3.drop("id",axis=1)], axis=1)

In [262]:
# df_dummy_backup = df_dummy.copy(deep=True)
# pd.concat([df_dummy.reset_index(drop=True),zip.drop("id",axis=1)])
# zip
# df_dummy.reset_index(drop=True).append(zip,axis=1)
zip.to_pickle('./zipcode_onehot.pkl')

In [ ]:
"""
#room_type　カテゴリ化、順序付けできそう
pd.get_dummies(df_dummy, columns=['room_type'])
df_dummy.head()
"""

In [14]:
# review_scores_rating　0.5区切りとかで集約
df_dummy["review_scores_rating"]=df["review_scores_rating"].fillna(df["review_scores_rating"].mean())
df_dummy["review_scores_rating"] = pd.cut(df_dummy["review_scores_rating"], 20, labels=[i*5 for i in range(20)])

In [15]:
# property_type 欠損値を0に、それ以外を1に
df_dummy["thumbnail_url"][df_dummy["thumbnail_url"].notnull()] = 1
df_dummy["thumbnail_url"] = df_dummy["thumbnail_url"].fillna(0)

In [271]:
df_dummy

,id,accommodates,amenities,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,thumbnail_url,zipcode,y
0,0,6,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",2.0,Real Bed,1.0,4.0,5,LA,t,My place is meant for family and a few friends...,2016-07-01,t,f,NaN,2016-07-01,f,2016-07-01,34.0,75,The Penthouse,NaN,1,Apartment,45,Private room,0,90804,138.0
1,1,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,3,DC,t,This is a new listing for a lovely guest bedro...,2016-09-01,t,t,100%,2015-12-01,f,2017-03-01,39.0,115,Guest Bedroom in Brookland,Brookland,9,House,95,Private room,1,20018,42.0
2,2,2,"{TV,Internet,""Wireless Internet"",Kitchen,""Indo...",2.0,Real Bed,1.0,1.0,3,NYC,t,We're looking forward to your stay at our apt....,2016-06-01,t,f,100%,2016-05-01,t,2017-08-01,41.0,119,Clean Modern Room in Lux Apt 1 Block From J Train,Bushwick,27,Apartment,75,Private room,1,NaN,65.0
3,3,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,3,SF,t,BEST CITY VIEWS - - ROOF DECK W/ BBQ & WiFi - ...,2014-03-01,t,t,100%,2012-06-01,t,2017-09-01,38.0,71,BEST views + reviews! 5/5 stars*****,Nob Hill,38,Apartment,90,Private room,0,94133,166.0
4,4,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,Real Bed,1.0,1.0,3,NYC,t,Charming Apartment on the upper west side of M...,2015-08-01,t,t,100%,2015-03-01,f,2017-09-01,41.0,119,Charming 1-bedroom - UWS Manhattan,Upper West Side,5,Apartment,95,Entire home/apt,1,10024,165.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18523,18523,4,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,Real Bed,2.0,2.0,3,NYC,t,"The Greenhouse, located on Green Street, is a ...",NaT,t,t,100%,2009-11-01,f,NaT,41.0,119,Spacious 2BR Greenpoint Getaway,Greenpoint,0,Apartment,90,Entire home/apt,1,11222,NaN
18524,18524,2,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",1.0,Real Bed,2.0,1.0,5,Chicago,f,"Two bedroom, one bathroom with large dining/li...",2017-01-01,t,f,100%,2017-01-01,f,2017-04-01,42.0,103,Walk up Apartment in Lakeview/Wrigleyville,Lakeview,9,Apartment,85,Entire home/apt,0,60657,NaN
18525,18525,5,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",1.5,Real Bed,2.0,2.0,5,Chicago,t,Happy Holidays! If you're looking for a big op...,NaT,t,f,100%,2014-09-01,f,NaT,42.0,103,Beautiful Logan Square Home,Avondale,0,House,90,Entire home/apt,1,60618,NaN
18526,18526,2,"{Internet,""Wireless Internet"",""Air conditionin...",1.0,Real Bed,1.0,2.0,3,NYC,t,This is a cozy one-bedroom apartment a few blo...,2016-04-01,t,f,100%,2014-03-01,f,2017-05-01,41.0,119,Charming 1 BR apartment east of Central Park,East Harlem,4,Apartment,90,Entire home/apt,1,10029.0,NaN


## **とりあえず欠損値を埋めてみる**

In [ ]:
#とりあえずの方向性
"""
bathrooms                   200 float 平均
bedrooms                     91 char 平均の方が良さそう
beds                        131 float 平均
host_response_rate        18299 char 平均
neighbourhood              6872 char 0でもいいかも(多分ないってことだから)
host_has_profile_pic        188 int 0.5?
host_identity_verified      188 int 0.5?

完了済：
review_scores_rating      16722 float null多すぎて検討つかないからとりあえず平均
thumbnail_url              8216 char 0
zipcode                     966 全部0
first_review              15864 char onehot全部0
host_since                  188 char onehot全部0
last_review               15827 char onehot全部0
平均値打ち込んだやつは全部新たなラベル1を付け加える(変更判定ラベル)
"""

In [17]:
df_dummy["host_response_rate"] = df_dummy["host_response_rate"].map(lambda x: int(x.replace("%","").strip()) if type(x)==str else x )

In [18]:
df_dummy["host_response_rate"]

0          NaN
1        100.0
2        100.0
3        100.0
4        100.0
         ...  
18523    100.0
18524    100.0
18525    100.0
18526    100.0
18527    100.0
Name: host_response_rate, Length: 74111, dtype: float64

In [16]:
#　平均処理
df_dummy["bathrooms"]=df_dummy["bathrooms"].fillna(df["bathrooms"].mean())
df_dummy["bedrooms"]=df_dummy["bedrooms"].fillna(df["bedrooms"].mean())
df_dummy["beds"]=df_dummy["beds"].fillna(df["beds"].mean())
df_dummy["host_response_rate"]=df_dummy["host_response_rate"].fillna("0")

#0埋め
df_dummy["neighbourhood"]=df_dummy["neighbourhood"].fillna("0")

In [17]:
# t-fを0-1に変える
df_dummy["cleaning_fee"]=df_dummy["cleaning_fee"].replace({"t":1, "f":0})
df_dummy["host_has_profile_pic"]=df_dummy["host_has_profile_pic"].replace({"t":1, "f":0})
df_dummy["host_identity_verified"]=df_dummy["host_identity_verified"].replace({"t":1, "f":0})
df_dummy["instant_bookable"]=df_dummy["instant_bookable"].replace({"t":1, "f":0})

In [18]:
# t-fのやつを0.5で埋める
df_dummy["host_has_profile_pic"]=df_dummy["host_has_profile_pic"].fillna(0.5)
df_dummy["host_identity_verified"]=df_dummy["host_identity_verified"].fillna(0.5)

In [19]:
# 上で変になってたやつ
df_dummy["first_review"]=df_dummy["first_review"].replace({"NaT":"0"})
df_dummy["last_review"]=df_dummy["last_review"].replace({"NaT":"0"})
df_dummy["host_since"]=df_dummy["host_since"].replace({"NaT":"0"})

In [20]:
df_dummy_train = df_dummy.copy(deep=True)
df_dummy_train = df_dummy_train.drop(["id","amenities","description","name","zipcode"], axis=1)

In [21]:
X_df_dummy_train=df_dummy_train[0:55583]
TARGET = df_dummy_train[55583:]

In [22]:
X_df_dummy_train.to_pickle('./train.pkl')
TARGET.to_pickle('./target.pkl')

In [54]:
X_df_dummy_train.isnull().sum()

accommodates              0
bathrooms                 0
bed_type                  0
bedrooms                  0
beds                      0
cancellation_policy       0
city                      0
cleaning_fee              0
first_review              0
host_has_profile_pic      0
host_identity_verified    0
host_response_rate        0
host_since                0
instant_bookable          0
last_review               0
latitude                  0
longitude                 0
neighbourhood             0
number_of_reviews         0
property_type             0
review_scores_rating      0
room_type                 0
thumbnail_url             0
y                         0
dtype: int64

# **とりあえずCatBoost**

In [35]:
def cross_validate(split_size=5):
  results = []
  kf = KFold(n_splits=split_size)
  for train_idx, val_idx in kf.split(X_train, y_train):
    train_x = X_train[train_idx]
    train_y = y_train[train_idx]
    val_x = X_train[val_idx]
    val_y = y_train[val_idx]
    rmse = run_catboost(train_x, train_y, val_x, val_y)
    results.append(rmse)
  return results

def run_catboost(x,y,val_x,val_y):
  # params = {
  #       # タスク設定と損失関数
  #       'loss_function': 'Logloss',
  #       # 学習ラウンド数
  #       'num_boost_round': 100,
  #   }
  # モデルを学習する
  categorical_features_indices = []
  for index, j in enumerate(X_train[0]):
    if (type(j) == str):
      categorical_features_indices.append(index)
  train_pool = Pool(x, label=y, cat_features=categorical_features_indices)
  test_pool = Pool(val_x, label=val_y, cat_features=categorical_features_indices)
  model = CatBoostRegressor(iterations=3000, learning_rate=0.05, depth=5)
  model.fit(train_pool)
  # 検証用データを分類する
  # NOTE: 確率がほしいときは prediction_type='Probability' を使う
  pred_y = model.predict(test_pool)
  # 精度 (Accuracy) を検証する
  rmse = np.sqrt(mean_squared_error(val_y, pred_y))
  print('RMSE:', rmse)
  return rmse

X_train = X_df_dummy_train.dropna().drop(["y"],axis=1).values
y_train = X_df_dummy_train.dropna()["y"].values
def_results = cross_validate(split_size=5)
print("results")
print(def_results)

CatBoostError: ignored

In [ ]:
def run_catboost(x,y,target):
  # params = {
  #       # タスク設定と損失関数
  #       'loss_function': 'Logloss',
  #       # 学習ラウンド数
  #       'num_boost_round': 100,
  #   }
  # モデルを学習する
  categorical_features_indices = []
  for index, j in enumerate(X_train[0]):
    if (type(j) == str):
      categorical_features_indices.append(index)
  train_pool = Pool(x, label=y, cat_features=categorical_features_indices)
  test_pool = Pool(target, cat_features=categorical_features_indices)
  model = CatBoostRegressor(iterations=3000, learning_rate=0.05, depth=5)
  model.fit(train_pool)
  # 検証用データを分類する
  # NOTE: 確率がほしいときは prediction_type='Probability' を使う
  pred_y = model.predict(test_pool)
  # 精度 (Accuracy) を検証する
  # rmse = np.sqrt(mean_squared_error(val_y, pred_y))
  # print('RMSE:', rmse)
  return pred_y

X_train = X_df_dummy_train.dropna().drop(["y"],axis=1).values
y_train = X_df_dummy_train.dropna()["y"].values
target = TARGET.dropna().drop(["y"],axis=1).values
predict = run_catboost(X_train, y_train, target)
print("results")
print(predict)

0:	learn: 164.8290629	total: 165ms	remaining: 8m 13s
1:	learn: 161.7235292	total: 321ms	remaining: 8m 1s
2:	learn: 158.7714496	total: 405ms	remaining: 6m 44s
3:	learn: 156.0853311	total: 490ms	remaining: 6m 6s
4:	learn: 153.5576468	total: 569ms	remaining: 5m 40s
5:	learn: 151.3304281	total: 657ms	remaining: 5m 27s
6:	learn: 149.1712928	total: 741ms	remaining: 5m 17s
7:	learn: 146.8877453	total: 831ms	remaining: 5m 10s
8:	learn: 144.7250010	total: 936ms	remaining: 5m 11s
9:	learn: 142.7561918	total: 1.04s	remaining: 5m 11s
10:	learn: 140.9590806	total: 1.12s	remaining: 5m 4s
11:	learn: 139.2544135	total: 1.2s	remaining: 4m 57s
12:	learn: 137.6548385	total: 1.26s	remaining: 4m 50s
13:	learn: 136.1336018	total: 1.34s	remaining: 4m 45s
14:	learn: 134.7160495	total: 1.43s	remaining: 4m 44s
15:	learn: 133.4066927	total: 1.5s	remaining: 4m 40s
16:	learn: 132.2426152	total: 1.58s	remaining: 4m 37s
17:	learn: 131.1608022	total: 1.68s	remaining: 4m 37s
18:	learn: 130.1465299	total: 1.75s	remaini

In [ ]:
with open('dict.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in mydict.items():
       writer.writerow([key, value])

In [26]:
!pip install optuna 

     |████████████████████████████████| 235kB 2.8MB/s 
     |████████████████████████████████| 1.1MB 9.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 81kB 7.1MB/s 
     |████████████████████████████████| 81kB 7.7MB/s 
     |████████████████████████████████| 133kB 25.7MB/s 
     |████████████████████████████████| 112kB 15.8MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
  Created wheel for alembic: filename=alembic-1.4.2-cp36-none-any.whl size=159540 sha256=83caa9de0b874fe7b6b9e4936ad8ce02bfb770310fbc61f7c105ea3e5ef94b33
  Stored in directory: /root/.cache/pip/wheels/1f/04/83/76023f7a4c14688c0b5c2682a96392cfdd3ee4449eaaa287ef
Successfully built alembic
  Created wheel for optuna: filename=optuna-2.0.0-cp36-none-any.whl size=312967 sha256=8028feb544c0ffb905d2e412e561efcb1f5ba664df08a7a6bbf2b0a635796de2
  Stored in directory: /root/.cache/pip/wheels/b5/

## **optuna-catboost**

In [29]:
from sklearn.model_selection import train_test_split
from catboost import Pool
import sklearn.metrics
def objective(trial):
    X_train = X_df_dummy_train.dropna().drop(["y"],axis=1).values
    y_train = .dropna()["y"].values
    # トレーニングデータとテストデータを分割
    categorical_features_indices = []
    for index, j in enumerate(X_train[0]):
      if (type(j) == str):
        categorical_features_indices.append(index)
      train_pool = Pool(X_train, label=y_train, cat_features=categorical_features_indices)
      test_pool = Pool(TARGET, label=test_y, cat_features=categorical_features_indices)

    # パラメータの指定
    params = {
        'iterations' : trial.suggest_int('iterations', 50, 300),                         
        'depth' : trial.suggest_int('depth', 4, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),               
        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), 
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait' :trial.suggest_int('od_wait', 10, 50)
    }

    # 学習
    model = CatBoostClassifier(**params)
    model.fit(train_pool)
    # 予測
    pred_y = model.predict(test_pool)
    pred_y = np.rint(pred_y)
    # 精度 (Accuracy) を検証する
    rmse = np.sqrt(mean_squared_error(val_y, pred_y))
    print('RMSE:', rmse)

    return rmse

import optuna
study = optuna.create_study()
study.optimize(objective, n_trials=10)
print(study.best_trial)

[W 2020-08-30 14:54:20,377] Trial 0 failed because of the following error: NameError("name 'train_x' is not defined",)
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/optuna/study.py", line 709, in _run_trial
    result = func(trial)
  File "<ipython-input-29-09fcfbb9b39d>", line 12, in objective
    train_pool = Pool(train_x, label=train_y, cat_features=categorical_features_indices)
NameError: name 'train_x' is not defined


NameError: ignored